In [1]:
import sys
import yaml
from dash import Dash, dcc, html, Input, Output, State
import plotly.express as px
import pandas as pd
from base64 import b64encode
import io
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from datetime import datetime

def figures_to_html(figs):
    current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"dashboard_{current_time}.html"
    with open(filename, 'w') as dashboard:
        dashboard.write("<html><head></head><body>" + "\n")
        for fig in figs:
            inner_html = fig.to_html(full_html=False, include_plotlyjs='cdn')#.split('<body>')[1].split('</body>')[0]
            dashboard.write(inner_html)
        dashboard.write("</body></html>" + "\n")



external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = Dash(__name__, external_stylesheets=external_stylesheets)


app.layout = html.Div([
    dcc.Upload(
        id='upload-data',
        children=html.Div([
            'Drag and Drop or ',
            html.A('Select Files')
        ]),
        style={
            'width': '100%',
            'height': '60px',
            'lineHeight': '60px',
            'borderWidth': '1px',
            'borderStyle': 'dashed',
            'borderRadius': '5px',
            'textAlign': 'center',
            'margin': '10px'
        },
        # Allow multiple files to be uploaded
        multiple=False
    ),
    html.Div(id='output-data-upload'),

])

def parse_contents(contents, filename, date):
    content_type, content_string = contents.split(',')

    decoded = base64.b64decode(content_string)
    try:
        if 'csv' in filename:
            # Assume that the user uploaded a CSV file
            df = pd.read_csv(
                io.StringIO(decoded.decode('utf-8')))
        elif 'xls' in filename:
            # Assume that the user uploaded an excel file
            df = pd.read_excel(io.BytesIO(decoded))
    except Exception as e:
        print(e)
        return html.Div([
            'There was an error processing this file.'
        ])

    return html.Div([
        html.H5(filename),
        html.H6(datetime.datetime.fromtimestamp(date)),

        dash_table.DataTable(
            df.to_dict('records'),
            [{'name': i, 'id': i} for i in df.columns]
        ),

        html.Hr(),  # horizontal line

        # For debugging, display the raw contents provided by the web browser
        html.Div('Raw Content'),
        html.Pre(contents[0:200] + '...', style={
            'whiteSpace': 'pre-wrap',
            'wordBreak': 'break-all'
        })
    ])

# @app.callback(Output('output-data-upload', 'children'),
#               Input('upload-data', 'contents'),
#               State('upload-data', 'filename'),
#               State('upload-data', 'last_modified'))
def update_output1(list_of_contents, list_of_names, list_of_dates):
    if list_of_contents is not None:
        children = [
            parse_contents(c, n, d) for c, n, d in
            zip(list_of_contents, list_of_names, list_of_dates)]
        return children


@app.callback(Output('output-data-upload', 'children'),
              Input('upload-data', 'contents'),
              State('upload-data', 'filename'),
              State('upload-data', 'last_modified'))
def update_output(list_of_contents, list_of_names, list_of_dates):
    if list_of_contents is not None:
        lane1_csv_file = list_of_names

        df1 = pd.read_csv(lane1_csv_file)
        print(f"File csv file: {lane1_csv_file}...")
        # print(df1['time'])

        my_fig1 = px.line(df1, x='time', y=df1.columns, markers=True, title=lane1_csv_file)
        my_fig1.update_layout(
            boxmode='group',
            plot_bgcolor='black',
            paper_bgcolor='white',
            xaxis=dict(showgrid=False),
            yaxis=dict(showgrid=False)
        )
        my_fig1.update_traces(visible='legendonly')

        fig = []
        fig.append(my_fig1)
        figures_to_html(fig)
    
        return html.Div([
            dcc.Graph(figure=my_fig1)
        ])

# @app.callback(
#     Output("graph", "figure"),
#     [Input("checklist_option", "value")])
def update_line_chart(userOption):

    lane1_csv_file = '20240915_210808.csv'

    df1 = pd.read_csv(lane1_csv_file)
    print(f"File csv file: {lane1_csv_file}...")

    
    my_fig1 = px.line(df1, x='time', y=df1.columns, markers=True, title=lane1_csv_file)
    my_fig1.update_layout(
        boxmode='group',
        plot_bgcolor='black',
        paper_bgcolor='white',
        xaxis=dict(showgrid=False),
        yaxis=dict(showgrid=False)
    )
    my_fig1.update_traces(visible='legendonly')
    return my_fig1
    
app.run_server(debug=True)

File csv file: 20240917_134405.csv...
---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
Cell In[1], line 126, in update_output(
    list_of_contents='data:text/csv;base64,dGltZSxtb3RvclBvd2VyLGJhdHR...3NjA3MzYwNTg4MjY1ODYsMTIuODM2MDEzNzk4NzMyMTg0DQo=',
    list_of_names='20240917_134405.csv',
    list_of_dates=1726605863.726
)
    124 fig = []
    125 fig.append(my_fig1)
--> 126 figures_to_html(fig)
        fig = [Figure({
    'data': [{'hovertemplate': 'variable=motorPower<br>time=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'motorPower',
              'line': {'color': '#636efa', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines+markers',
              'name': 'motorPower',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'visible': 'legendonly',
          